## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Quatre Cocos,MU,2021-11-07 04:23:00,-20.2078,57.7625,81.09,77,43,14.41,scattered clouds
1,1,Kahului,US,2021-11-07 04:23:00,20.8947,-156.4700,78.82,73,1,23.02,clear sky
2,2,Busselton,AU,2021-11-07 04:23:01,-33.6500,115.3333,66.43,41,44,4.23,scattered clouds
3,3,Tiksi,RU,2021-11-07 04:23:01,71.6872,128.8694,0.63,87,100,15.86,overcast clouds
4,4,Livingston,GB,2021-11-07 04:18:59,55.9029,-3.5226,46.76,72,75,16.11,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Quatre Cocos,MU,2021-11-07 04:23:00,-20.2078,57.7625,81.09,77,43,14.41,scattered clouds
1,1,Kahului,US,2021-11-07 04:23:00,20.8947,-156.4700,78.82,73,1,23.02,clear sky
16,16,Karratha,AU,2021-11-07 04:23:05,-20.7377,116.8463,84.63,43,7,16.62,clear sky
18,18,Kavaratti,IN,2021-11-07 04:23:06,10.5669,72.6420,81.81,73,96,9.04,overcast clouds
21,21,Mazatlan,MX,2021-11-07 04:23:08,23.2167,-106.4167,81.01,90,21,1.99,few clouds
24,24,Avarua,CK,2021-11-07 04:23:09,-21.2078,-159.7750,75.25,73,75,11.50,broken clouds
25,25,Aitape,PG,2021-11-07 04:23:09,-3.1374,142.3475,86.81,63,78,7.74,broken clouds
27,27,Fare,PF,2021-11-07 04:23:10,-16.7000,-151.0167,80.06,69,32,5.84,scattered clouds
30,30,Sabang,ID,2021-11-07 04:23:11,5.8933,95.3214,79.79,79,100,3.65,overcast clouds
32,32,Hilo,US,2021-11-07 04:23:11,19.7297,-155.0900,79.29,71,90,3.44,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                245
City                   245
Country                242
Date                   245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()


In [7]:
new_preferred_cities_df.count()

City_ID                242
City                   242
Country                242
Date                   242
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,81.09,scattered clouds,-20.2078,57.7625,
1,Kahului,US,78.82,clear sky,20.8947,-156.4700,
16,Karratha,AU,84.63,clear sky,-20.7377,116.8463,
18,Kavaratti,IN,81.81,overcast clouds,10.5669,72.6420,
21,Mazatlan,MX,81.01,few clouds,23.2167,-106.4167,
24,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
25,Aitape,PG,86.81,broken clouds,-3.1374,142.3475,
27,Fare,PF,80.06,scattered clouds,-16.7000,-151.0167,
30,Sabang,ID,79.79,overcast clouds,5.8933,95.3214,
32,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
 
    # 6c. Get latitude and longitude from DataFram
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
  
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
#     print(hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,81.09,scattered clouds,-20.2078,57.7625,Sunrise Attitude Hotel
1,Kahului,US,78.82,clear sky,20.8947,-156.4700,Maui Seaside Hotel
16,Karratha,AU,84.63,clear sky,-20.7377,116.8463,ibis Styles Karratha
18,Kavaratti,IN,81.81,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
21,Mazatlan,MX,81.01,few clouds,23.2167,-106.4167,Hotel Playamar
24,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
25,Aitape,PG,86.81,broken clouds,-3.1374,142.3475,
27,Fare,PF,80.06,scattered clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
30,Sabang,ID,79.79,overcast clouds,5.8933,95.3214,Hotel Citra
32,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,81.09,scattered clouds,-20.2078,57.7625,Sunrise Attitude Hotel
1,Kahului,US,78.82,clear sky,20.8947,-156.4700,Maui Seaside Hotel
16,Karratha,AU,84.63,clear sky,-20.7377,116.8463,ibis Styles Karratha
18,Kavaratti,IN,81.81,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
21,Mazatlan,MX,81.01,few clouds,23.2167,-106.4167,Hotel Playamar
24,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
27,Fare,PF,80.06,scattered clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
30,Sabang,ID,79.79,overcast clouds,5.8933,95.3214,Hotel Citra
32,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
33,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,Villa Enata


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))